## Worldometer Site Web Scrapping

---

In [1]:
# Import libraries

import os
import re
import glob
import requests 
import numpy as np
import pandas as pd
from datetime import datetime
from bs4 import BeautifulSoup

In [2]:
# Import dataset

link = 'https://www.worldometers.info/coronavirus/'
req = requests.get(link)
soup = BeautifulSoup(req.content, "html.parser")

In [3]:
# Find table

thead = soup.find_all('thead')[-1]
head = thead.find_all('tr')
tbody = soup.find_all('tbody')[0]
body = tbody.find_all('tr')

In [4]:
# Get the table contents

head_rows = []
body_rows = []

for tr in head:
    td = tr.find_all(['th', 'td'])
    row = [i.text for i in td]
    head_rows.append(row)
# print(head_rows)

for tr in body:
    td = tr.find_all(['th', 'td'])
    row = [i.text for i in td]
    body_rows.append(row)
print(head_rows)

[['#', 'Country,Other', 'TotalCases', 'NewCases', 'TotalDeaths', 'NewDeaths', 'TotalRecovered', 'NewRecovered', 'ActiveCases', 'Serious,Critical', 'Tot\xa0Cases/1M pop', 'Deaths/1M pop', 'TotalTests', 'Tests/\n1M pop\n', 'Population', 'Continent', '1 Caseevery X ppl', '1 Deathevery X ppl', '1 Testevery X ppl']]


In [5]:
# Save content in dataframe

df_wm = pd.DataFrame(body_rows[:len(body_rows)-6], 
                     columns=head_rows[0])         

df_wm.tail(5)

,#,"Country,Other",TotalCases,NewCases,TotalDeaths,NewDeaths,TotalRecovered,NewRecovered,ActiveCases,"Serious,Critical",Tot Cases/1M pop,Deaths/1M pop,TotalTests,Tests/\n1M pop\n,Population,Continent,1 Caseevery X ppl,1 Deathevery X ppl,1 Testevery X ppl
217,210,Falkland Islands,16,,,,13,,3,,"4,544",,"4,245","1,205,623","3,521",South America,220,,1
218,211,Saint Pierre Miquelon,16,,,,12,,4,,"2,767",,"2,814","486,683","5,782",North America,361,,2
219,212,Montserrat,13,,1,,12,,0,,"2,604",200,577,"115,562","4,993",North America,384,"4,993",9
220,213,Western Sahara,10,,1,,8,,1,,17,2,,,"603,270",Africa,"60,327","603,270",
221,214,MS Zaandam,9,,2,,,,7,,,,,,,,,,


In [6]:
# Drop unwanted rows & columns

df_wm = df_wm.iloc[8:, :-3].reset_index(drop=True)
df_wm = df_wm.drop('#', axis=1)
df_wm.head()

,"Country,Other",TotalCases,NewCases,TotalDeaths,NewDeaths,TotalRecovered,NewRecovered,ActiveCases,"Serious,Critical",Tot Cases/1M pop,Deaths/1M pop,TotalTests,Tests/\n1M pop\n,Population,Continent
0,USA,"13,746,232","+134,016","273,064",+811,"8,104,684","+63,445","5,368,484","24,822","41,429",823,"192,267,997","579,467","331,801,570",North America
1,India,"9,432,039","+39,000","137,177",+444,"8,846,187","+45,026","448,675","8,944","6,807",99,"139,503,803","100,684","1,385,567,591",Asia
2,Brazil,"6,314,740","+24,468","172,833",+196,"5,578,118","+15,579","563,789","8,318","29,622",811,"21,900,000","102,730","213,180,600",South America
3,Russia,"2,269,316","+26,683","39,527",+459,"1,761,457","+21,987","468,332","2,300","15,547",271,"75,900,000","520,004","145,960,348",Europe
4,France,"2,218,483","+9,784","52,325",+198,"161,427",+290,"2,004,731","3,756","33,956",801,"20,512,082","313,961","65,333,164",Europe


In [7]:
# Rename & rearrange columns
df_wm.columns = ['Country/Region', 'TotalCases', 'NewCases', 'TotalDeaths', 'NewDeaths',
       'TotalRecovered', 'NewRecovered', 'ActiveCases', 'Serious,Critical',
       'Tot Cases/1M pop', 'Deaths/1M pop', 'TotalTests', 'Tests/1M pop',
       'Population', 'Continent']

df_wm = df_wm[['Country/Region', 'Continent', 'Population', 'TotalCases', 'NewCases', 'TotalDeaths', 'NewDeaths',
       'TotalRecovered', 'NewRecovered', 'ActiveCases', 'Serious,Critical',
       'Tot Cases/1M pop', 'Deaths/1M pop', 'TotalTests', 'Tests/1M pop' ]]

df_wm.head()

,Country/Region,Continent,Population,TotalCases,NewCases,TotalDeaths,NewDeaths,TotalRecovered,NewRecovered,ActiveCases,"Serious,Critical",Tot Cases/1M pop,Deaths/1M pop,TotalTests,Tests/1M pop
0,USA,North America,"331,801,570","13,746,232","+134,016","273,064",+811,"8,104,684","+63,445","5,368,484","24,822","41,429",823,"192,267,997","579,467"
1,India,Asia,"1,385,567,591","9,432,039","+39,000","137,177",+444,"8,846,187","+45,026","448,675","8,944","6,807",99,"139,503,803","100,684"
2,Brazil,South America,"213,180,600","6,314,740","+24,468","172,833",+196,"5,578,118","+15,579","563,789","8,318","29,622",811,"21,900,000","102,730"
3,Russia,Europe,"145,960,348","2,269,316","+26,683","39,527",+459,"1,761,457","+21,987","468,332","2,300","15,547",271,"75,900,000","520,004"
4,France,Europe,"65,333,164","2,218,483","+9,784","52,325",+198,"161,427",+290,"2,004,731","3,756","33,956",801,"20,512,082","313,961"


In [8]:
who_region = {}

# African Region AFRO
afro = "Algeria, Angola, Cabo Verde, Congo, DRC, Eswatini, Sao Tome and Principe, Benin, South Sudan, Western Sahara, Congo (Brazzaville), Congo (Kinshasa), Cote d'Ivoire, Botswana, Burkina Faso, Burundi, Cameroon, Cape Verde, Central African Republic, Chad, Comoros, Ivory Coast, Democratic Republic of the Congo, Equatorial Guinea, Eritrea, Ethiopia, Gabon, Gambia, Ghana, Guinea, Guinea-Bissau, Kenya, Lesotho, Liberia, Madagascar, Malawi, Mali, Mauritania, Mauritius, Mozambique, Namibia, Niger, Nigeria, Republic of the Congo, Rwanda, São Tomé and Príncipe, Senegal, Seychelles, Sierra Leone, Somalia, South Africa, Swaziland, Togo, Uganda, Tanzania, Zambia, Zimbabwe"
afro = [i.strip() for i in afro.split(',')]
for i in afro:
    who_region[i] = 'Africa'
    
# Region of the Americas PAHO
paho = 'Antigua and Barbuda, Argentina, Bahamas, Barbados, Belize, Bermuda, Bolivia, Brazil, Canada, Chile, Colombia, Costa Rica, Cuba, Dominica, Dominican Republic, Ecuador, El Salvador, Grenada, Guatemala, Guyana, Haiti, Honduras, Jamaica, Mexico, Nicaragua, Panama, Paraguay, Peru, Saint Kitts and Nevis, Saint Lucia, Saint Vincent and the Grenadines, Suriname, Trinidad and Tobago, United States, US, USA, Uruguay, Venezuela'
paho = [i.strip() for i in paho.split(',')]
for i in paho:
    who_region[i] = 'Americas'

# South-East Asia Region SEARO
searo = 'Bangladesh, Bhutan, North Korea, India, Indonesia, Maldives, Myanmar, Burma, Nepal, Sri Lanka, Thailand, Timor-Leste'
searo = [i.strip() for i in searo.split(',')]
for i in searo:
    who_region[i] = 'South-East Asia'

# European Region EURO
euro = 'Albania, Andorra, Greenland, Kosovo, Holy See, Vatican City, Liechtenstein, Armenia, Czechia, Austria, Azerbaijan, Belarus, Belgium, Bosnia and Herzegovina, Bulgaria, Croatia, Cyprus, Czech Republic, Denmark, Estonia, Finland, France, Georgia, Germany, Greece, Hungary, Iceland, Ireland, Israel, Italy, Kazakhstan, Kyrgyzstan, Latvia, Lithuania, Luxembourg, Malta, Monaco, Montenegro, Netherlands, North Macedonia, Norway, Poland, Portugal, Moldova, Romania, Russia, San Marino, Serbia, Slovakia, Slovenia, Spain, Sweden, Switzerland, Tajikistan, Turkey, Turkmenistan, Ukraine, United Kingdom, UK, Uzbekistan'
euro = [i.strip() for i in euro.split(',')]
for i in euro:
    who_region[i] = 'Europe'

# Eastern Mediterranean Region EMRO
emro = 'Afghanistan, Bahrain, Djibouti, Egypt, Iran, Iraq, Jordan, Kuwait, Lebanon, Libya, Morocco, Oman, Pakistan, Palestine, West Bank and Gaza, Qatar, Saudi Arabia, Somalia, Sudan, Syria, Tunisia, United Arab Emirates, UAE, Yemen'
emro = [i.strip() for i in emro.split(',')]
for i in emro:
    who_region[i] = 'Eastern Mediterranean'

# Western Pacific Region WPRO
wpro = 'Australia, Brunei, Cambodia, China, Cook Islands, Fiji, Japan, Hong Kong, Kiribati, Laos, Malaysia, Marshall Islands, Micronesia, Mongolia, Nauru, New Zealand, Niue, Palau, Papua New Guinea, Philippines, South Korea, S. Korea, Samoa, Singapore, Solomon Islands, Taiwan, Taiwan*, Tonga, Tuvalu, Vanuatu, Vietnam'
wpro = [i.strip() for i in wpro.split(',')]
for i in wpro:
    who_region[i] = 'Western Pacific'

In [9]:
# Addding 'WHO Region' & list of countries with missing "WHO Region"

df_wm['WHO Region'] = df_wm['Country/Region'].map(who_region)
df_wm[df_wm['WHO Region'].isna()]['Country/Region'].unique()

array(['French Polynesia', 'French Guiana', 'Guadeloupe', 'Réunion',
       'Martinique', 'Mayotte', 'CAR', 'Aruba', 'Curaçao',
       'Channel Islands', 'Sint Maarten', 'Gibraltar', 'Turks and Caicos',
       'Diamond Princess', 'Saint Martin', 'Faeroe Islands',
       'Isle of Man', 'Cayman Islands', 'Caribbean Netherlands',
       'Brunei ', 'St. Barth', 'St. Vincent Grenadines',
       'British Virgin Islands', 'Macao', 'New Caledonia',
       'Falkland Islands', 'Saint Pierre Miquelon', 'Montserrat',
       'MS Zaandam'], dtype=object)

In [10]:
# Data corrections

for col in df_wm.columns[2:]:
    # replace comma with empty string
    df_wm[col] = df_wm[col].str.replace('[,+ ]', '', regex=True)
    # replace 'N/A' with empty string
    df_wm[col] = df_wm[col].str.replace('N/A', '', regex=False)

# replace empty strings with np.nan
df_wm = df_wm.replace('', np.nan)

df_wm.head()

,Country/Region,Continent,Population,TotalCases,NewCases,TotalDeaths,NewDeaths,TotalRecovered,NewRecovered,ActiveCases,"Serious,Critical",Tot Cases/1M pop,Deaths/1M pop,TotalTests,Tests/1M pop,WHO Region
0,USA,North America,331801570,13746232,134016,273064,811,8104684,63445,5368484,24822,41429,823,192267997,579467,Americas
1,India,Asia,1385567591,9432039,39000,137177,444,8846187,45026,448675,8944,6807,99,139503803,100684,South-EastAsia
2,Brazil,South America,213180600,6314740,24468,172833,196,5578118,15579,563789,8318,29622,811,21900000,102730,Americas
3,Russia,Europe,145960348,2269316,26683,39527,459,1761457,21987,468332,2300,15547,271,75900000,520004,Europe
4,France,Europe,65333164,2218483,9784,52325,198,161427,290,2004731,3756,33956,801,20512082,313961,Europe


In [11]:
# Save as "csv" file

df_wm.to_csv('worldometer_data.csv', index=False)